In [1]:
import pandas as pd
import numpy as np
import warnings
import scipy as sp
from datetime import datetime
from IPython.display import display
import plotly
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
from IPython.core.display import display, HTML
from plotly.subplots import make_subplots
from IPython.display import display
autoscroll_set_on_load = True
autoscroll_starting_threshold = -1
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df = pd.read_excel('data/LIGHT4DEAF_Neurocognition_Avril_2020_1_FULL_ADULTES_20200504.xlsx')

In [85]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [86]:
df = df[df["Investigateur"].notnull()]

In [3]:
cardinal = len(df)

In [4]:
now = datetime.now()
df['Date de naissance du patient'] = pd.to_datetime(df['Date de naissance du patient'], format='%m/%d/%y')
df['Date de naissance du patient'] = df['Date de naissance du patient'].where(df['Date de naissance du patient'] < now, df['Date de naissance du patient'] -  np.timedelta64(100, 'Y')) 
df['age'] = (now - df['Date de naissance du patient']).astype('<m8[Y]')

In [5]:
#Patients adultes : type
df_type1 = df[df["Type"]=='Type 1']
df_type2 = df[df["Type"]=='Type 2']
df_type3 = df[df["Type"]=='Type 3']
df_type_nan = df[df["Type"].isna()]

In [6]:
display(HTML('<h1>L i g h t 4 d e a f</h1>'))
display(HTML('<h2>Neurocognition - STROOP</h2>'))
display(HTML("<h3>Description de l'échantillon</h3>"))
display(HTML("<h4>1. Données : </h4>"))
display(HTML("<p>Enregistrements sélectionner sur 'STROOP' =. 'Oui' : </p>"))


In [7]:
cols = df.columns.values.tolist()
cols.index('STROOP')
cols.index('Autre commentaire')
arr = cols[cols.index('STROOP'):cols.index('Autre commentaire STROOP')]
arr.insert(0, "ID_SECONDAIRE")
arr.insert(1, "age")
arr.insert(2, "Type")
arr.insert(3, "Correction visuelle")
arr.insert(4, "Prothèse auditive DROITE")
arr.insert(5, "Prothèse auditive GAUCHE")
arr.insert(6, "Implant cochléaire DROIT")
arr.insert(7, "Implant cochléaire GAUCHE")
arr.insert(8, "Interprète LSF")
arr.insert(9, "Nombre d'années d'études")

In [8]:
df_str = df.loc[:,arr]

In [9]:
#Eliminitation du patient 8A A0 53 AC 37 comme demandé
df_str = df_str[df_str['ID_SECONDAIRE']!="8A A0 53 AC 37"]

In [10]:
df_str.loc[df_str['Type']=="Type 2", 'Type_color']="#B3D9FF"
df_str.loc[df_str['Type']=="Type 1", 'Type_color']="#33ffc9"
df_str.loc[df_str['Type']=="Type 3", 'Type_color']="#FFC433"
df_str.loc[df_str['Type'].isna(), 'Type_color']="#C5C5C5"
df_str['Type'].fillna('Type inconnu', inplace= True)

In [11]:
df_str = df_str[df_str['STROOP']=='Oui']
str_total = len(df_str[df_str['STROOP']=='Oui'])
str_age_avg = round(df_str["age"].mean())
str_age_min = df_str["age"].min()
str_age_max = df_str["age"].max()

In [12]:
#Patients ayant une aide auditive
df_str_coaud = df_str[(df_str["Prothèse auditive DROITE"]!="Aucune") | (df_str["Prothèse auditive GAUCHE"]!="Aucune") | (df_str["Implant cochléaire DROIT"]!="Aucun") | (df_str["Implant cochléaire GAUCHE"]!="Aucun")]
#Patients ayant uniquement une aide auditive
df_str_coaud_u = df_str_coaud[df_str_coaud["Correction visuelle"]=="Aucune"]
str_coaud_u = len(df_str_coaud_u)

In [78]:
#Patients ayant une correction visuelle
df_str_covis = df_str[df_str["Correction visuelle"]!="Aucune"]
#Patients ayant uniquement une correction visuelle
df_str_covis_u = df_str_covis[((df_str_covis["Prothèse auditive DROITE"]=="Aucune") & (df_str_covis["Prothèse auditive GAUCHE"]=="Aucune") & (df_str_covis["Implant cochléaire DROIT"]=="Aucun") & (df_str_covis["Implant cochléaire GAUCHE"]=="Aucun"))]
str_covis_u = len(df_str_covis_u)

In [79]:
#Patients ayant une aide auditive + visuelle
df_str_coaud_covis = df_str_coaud[df_str_coaud["Correction visuelle"]!="Aucune"]
str_coaud_covis = len(df_str_coaud_covis)

In [80]:
str_intlsf = len(df_str[df_str["Interprète LSF"]=="Oui"])

In [81]:
df_str.style

,ID_SECONDAIRE,age,Type,Correction visuelle,Prothèse auditive DROITE,Prothèse auditive GAUCHE,Implant cochléaire DROIT,Implant cochléaire GAUCHE,Interprète LSF,Nombre d'années d'études,STROOP,Réalisé en date du.1,"Si non, pourquoi.1",Temps dénomination pour 100 items,Ecart type - Temps dénomination pour 100 items,Rang percentile - Temps dénomination pour 100 items,Nombre d'erreurs non corrigés (100 items),Ecart type - Nombre d'erreurs non corrigés (100 items),Rang percentile - Nombre d'erreurs non corrigés (100 items),Temps de lecture,Ecart type - Temps de lecture,Rang percentile - Temps de lecture,Nombre d'erreurs (Temps de lecture),Ecart type - Nombre d'erreurs (Temps de lecture),Rang percentile - Nombre d'erreurs (Temps de lecture),Temps interférence,Ecart type - Temps interférence,Rang percentile - Temps interférence,Nombre d’erreurs non corrigées (Temps interférence),Ecart type - Nombre d’erreurs non corrigées (Temps interférence),Rang percentile - Nombre d’erreurs non corrigées (Temps interférence),Temps Int-déno,Ecart type - Temps Int-déno,Rang percentile - Temps Int-déno,Nombre d’erreurs non corrigées (Int-déno),Ecart type - Nombre d’erreurs non corrigées (Int-déno),Rang percentile - Nombre d’erreurs non corrigées (Int-déno),Commentaire,Type_color
8,39 CA BD 3C 82,22,Type 1,Verres correcteurs,Aucune,Aucune,Aucun,Implant cochléaire,Non,11,Oui,2018-04-06 00:00:00,nan,89,2.82,95,0,nan,nan,56,1.75,92,0,nan,nan,149,1.58,92,1,0.58,75,60,0.79,81,1,0.79,81,nan,#33ffc9
14,51 B7 6E C2 FD,40,Type 2,Verres correcteurs,Aucune,Aucune,Implant cochléaire,Implant cochléaire,Non,17,Oui,2017-11-21 00:00:00,nan,51,-0.55,32,0,nan,nan,40,0,54,0,nan,nan,73,-1.05,13,0,nan,nan,22,-1,17,0,nan,nan,nan,#B3D9FF
20,64 0B 1B 0A 45,38,Type 2,Aucune,Aucune,Aucune,Implant cochléaire,Implant cochléaire,Non,17,Oui,2017-12-12 00:00:00,nan,55,-0.011,46,0,nan,nan,48,1.33,90,0,nan,nan,75,0.94,17,2,3.16,95,20,-1.14,10,2,3.08,95,nan,#B3D9FF
26,EF 2E 22 E0 EA,18,Type 1,Refus de porter verres correcteurs ce jour,Aucune,Aucune,Implant cochléaire,Aucun,Oui,10,Oui,2018-06-22 00:00:00,nan,80,2,94,0,nan,nan,75,4.12,95,0,nan,nan,101,0.03,59,1,0.58,75,26,-0.62,17,1,0.79,81,nan,#33ffc9
30,8A 77 B8 09 AD,38,Type 2,Verres correcteurs,Prothèse auditive conventionnelle,Prothèse auditive conventionnelle,Aucun,Aucun,Non,17,Oui,2018-06-08 00:00:00,nan,70,1.5,91,0,nan,5,52,2,94,0,nan,5,98,0.3,70,1,1.4,90,28,-0.6,33,1,1.4,90,nan,#B3D9FF
34,90 01 09 71 F0,26,Type 2,Verres correcteurs,Prothèse auditive conventionnelle,Prothèse auditive conventionnelle,Aucun,Aucun,Non,11,Oui,2018-08-22 00:00:00,nan,73,1.36,87,0,nan,nan,50,1,86,0,nan,nan,128,0.9,88,0,nan,nan,55,0.58,77,0,nan,nan,nan,#B3D9FF
46,76 A2 26 6F C7,20,Type inconnu,Verres correcteurs,Aucune,Aucune,Implant cochléaire,Implant cochléaire,Non,11,Oui,2018-01-16 00:00:00,nan,80,2.66,95,0,nan,nan,67,4.5,95,0,nan,nan,117,1.38,86,2,3.16,95,37,0.07,57,2,3.08,95,nan,#C5C5C5
54,3F 34 F5 32 80,47,Type inconnu,Verres correcteurs,Prothèse auditive conventionnelle,Prothèse auditive conventionnelle,Aucun,Aucun,Non,14,Oui,2017-10-10 00:00:00,nan,63,0.7,77,0,nan,nan,57,2.66,95,0,nan,nan,137,1.36,92,0,nan,nan,74,1.27,92,0,nan,nan,nan,#C5C5C5
60,A6 FD C4 C0 69,48,Type 2,Verres correcteurs,Prothèse auditive conventionnelle,Prothèse auditive conventionnelle,Aucun,Aucun,Non,12,Oui,2017-07-18 00:00:00,nan,69,1.3,87,0,nan,nan,52,1.83,95,0,nan,nan,124,0.84,88,0,nan,nan,55,0.41,70,0,nan,nan,nan,#B3D9FF
68,7B DE 78 5B F6,23,Type 1,Verres correcteurs,Aucune,Aucune,Implant cochléaire,Aucun,Non,15,Oui,2018-07-13 00:00:00,nan,66,1.1,84,0,-0.2,5,51,1.8,93,0,-0.1,5,108,0.9,80,0,-0.5,5,42,0.4,50,0,-0.4,5,nan,#33ffc9


In [82]:
display(HTML('<h4>2. Effectifs </h4>'))

In [83]:
columns= ['','Valeur']
myarr = [["Effectif total (STROOP = Oui)",
            'Moyenne d’âge',
            'Âge minimum',
            'Âge maximum',
            'Patients ayant uniquement une correction visuelle',
            'Patients ayant uniquement une correction auditive',
            'Patients ayant une correction auditive + une correction visuelle',
            'Nombre de personnes ayant recours un interprète']]
for i in columns[1:]:
    myarr.append([str_total, str_age_avg, str_age_min, str_age_max, str_covis_u, str_coaud_u, str_coaud_covis, str_intlsf])

In [84]:
str_data =  go.Table(
                columnwidth = [30,5],
                header=dict(values=list(columns),
                align='left'),
                cells=dict(values= myarr,align='left'))
fig = go.Figure(data=str_data)
fig.show()

###
display(HTML("<p>Répartition du nombre d'années d'études :</p>"))
ranges = [0,5,10,12,15,20]
df_pop = df_str[df_str['STROOP']== 'Oui']

df_study = df_pop["Nombre d'années d'études"].copy()

df_study = df_study.groupby(pd.cut(df_pop["Nombre d'années d'études"], ranges, include_lowest=True)).count()
df_study = pd.DataFrame(df_study)
df_study = df_study.rename(columns={"Nombre d'années d'études":'Effectif'})
year_study = ["0 à 5", "5 à 10", "10 à 12", "12 à 15", "15 à 20"]
df_study["Nombre d'années d'études"] = year_study
df_study = df_study[["Nombre d'années d'études", "Effectif"]]
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_study.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_study["Effectif"], df_study["Nombre d'années d'études"]],
               fill_color='lavender',
               align='left'))
])

fig.show()

fig = px.bar(df_study, x="Nombre d'années d'études", y="Effectif", color="Effectif")
fig.update_layout(showlegend=False)
fig.show()

In [48]:
display(HTML("<h3>Scores temps</h3>"))
display(HTML('<h4>1. Temps dénomination pour 100 items</h4>'))

df_deno=df_str[df_str["Temps dénomination pour 100 items"].notna()|(df_str["Temps dénomination pour 100 items"]!=0)]
nb = len(df_deno)

nb_avg = round(df_deno["Temps dénomination pour 100 items"].mean())
nb_med = round(df_deno["Temps dénomination pour 100 items"].median())
nb_var = round(df_deno["Temps dénomination pour 100 items"].var())
nb_min = df_deno["Temps dénomination pour 100 items"].min()
nb_max = df_deno["Temps dénomination pour 100 items"].max()
#
ec_avg = round(df_deno["Ecart type - Temps dénomination pour 100 items"].mean())
ec_med = round(df_deno["Ecart type - Temps dénomination pour 100 items"].median())
ec_var = round(df_deno["Ecart type - Temps dénomination pour 100 items"].var())
ec_min = df_deno["Ecart type - Temps dénomination pour 100 items"].min()
ec_max = df_deno["Ecart type - Temps dénomination pour 100 items"].max()
#
rp_avg = round(df_deno["Rang percentile - Temps dénomination pour 100 items"].mean())
rp_med = round(df_deno["Rang percentile - Temps dénomination pour 100 items"].median())
rp_var = round(df_deno["Rang percentile - Temps dénomination pour 100 items"].var())
rp_min = df_deno["Rang percentile - Temps dénomination pour 100 items"].min()
rp_max = df_deno["Rang percentile - Temps dénomination pour 100 items"].max()

columns = ['',"Score", "Ecart-type", "Rang percentile"]

myarr = [['Effectif', 'Moyenne', 'Médiane', 'Variance', 'Min.', 'Max']]
myarr.append([nb, nb_avg, nb_med, nb_var, nb_min, nb_max])
myarr.append(['-',ec_avg, ec_med, ec_var, ec_min, ec_max])
myarr.append(['-',rp_avg, rp_med, rp_var, rp_min, rp_max])

myarr

tab =  go.Table(
                columnwidth = [15,15],
                header=dict(values=list(columns),
                align='left'),
                cells=dict(values= myarr,align='left'))
fig = go.Figure(data=tab)
fig.show()

In [49]:
display(HTML("<h4>2. Temps de lecture</h4>"))

df_deno=df_str[df_str["Temps de lecture"].notna()&(df_str["Temps de lecture"]!=0)]
nb = len(df_deno)
if nb != 0:
    nb_avg = round(df_deno["Temps de lecture"].mean())
    nb_med = round(df_deno["Temps de lecture"].median())
    nb_var = round(df_deno["Temps de lecture"].var())
    nb_min = df_deno["Temps de lecture"].min()
    nb_max = df_deno["Temps de lecture"].max()
    #
    ec_avg = round(df_deno["Ecart type - Temps de lecture"].mean())
    ec_med = round(df_deno["Ecart type - Temps de lecture"].median())
    ec_var = round(df_deno["Ecart type - Temps de lecture"].var())
    ec_min = df_deno["Ecart type - Temps de lecture"].min()
    ec_max = df_deno["Ecart type - Temps de lecture"].max()
    #
    rp_avg = round(df_deno["Rang percentile - Temps de lecture"].mean())
    rp_med = round(df_deno["Rang percentile - Temps de lecture"].median())
    rp_var = round(df_deno["Rang percentile - Temps de lecture"].var())
    rp_min = df_deno["Rang percentile - Temps de lecture"].min()
    rp_max = df_deno["Rang percentile - Temps de lecture"].max()
else:
    nb_avg = nb_med = nb_var = nb_min = nb_max = ec_avg = ec_med = ec_var = ec_min = ec_max = rp_avg= rp_med = rp_var = rp_min = rp_max =0
columns = ['',"Score", "Ecart-type", "Rang percentile"]

myarr = [['Effectif', 'Moyenne', 'Médiane', 'Variance', 'Min.', 'Max']]
myarr.append([nb, nb_avg, nb_med, nb_var, nb_min, nb_max])
myarr.append(['-',ec_avg, ec_med, ec_var, ec_min, ec_max])
myarr.append(['-',rp_avg, rp_med, rp_var, rp_min, rp_max])



tab =  go.Table(
                columnwidth = [15,15],
                header=dict(values=list(columns),
                align='left'),
                cells=dict(values= myarr,align='left'))
fig = go.Figure(data=tab)
fig.show()

In [22]:
display(HTML("<h4>3. Temps interférence</h4>"))

df_deno=df_str[df_str["Temps interférence"].notna()&(df_str["Temps interférence"]!=0)]
nb = len(df_deno)
if nb > 0:
    nb_avg = round(df_deno["Temps interférence"].mean())
    nb_med = round(df_deno["Temps interférence"].median())
    nb_var = round(df_deno["Temps interférence"].var())
    nb_min = df_deno["Temps interférence"].min()
    nb_max = df_deno["Temps interférence"].max()
    #
    ec_avg = round(df_deno["Ecart type - Temps interférence"].mean())
    ec_med = round(df_deno["Ecart type - Temps interférence"].median())
    ec_var = round(df_deno["Ecart type - Temps interférence"].var())
    ec_min = df_deno["Ecart type - Temps interférence"].min()
    ec_max = df_deno["Ecart type - Temps interférence"].max()
    #
    rp_avg = round(df_deno["Rang percentile - Temps interférence"].mean())
    rp_med = round(df_deno["Rang percentile - Temps interférence"].median())
    rp_var = round(df_deno["Rang percentile - Temps interférence"].var())
    rp_min = df_deno["Rang percentile - Temps interférence"].min()
    rp_max = df_deno["Rang percentile - Temps interférence"].max()
else:
    nb_avg = nb_med = nb_var = nb_min = nb_max = ec_avg = ec_med = ec_var = ec_min = ec_max = rp_avg= rp_med = rp_var = rp_min = rp_max=0
columns = ['',"Score", "Ecart-type", "Rang percentile"]

myarr = [['Effectif', 'Moyenne', 'Médiane', 'Variance', 'Min.', 'Max']]
myarr.append([nb, nb_avg, nb_med, nb_var, nb_min, nb_max])
myarr.append(['-',ec_avg, ec_med, ec_var, ec_min, ec_max])
myarr.append(['-',rp_avg, rp_med, rp_var, rp_min, rp_max])


tab =  go.Table(
                columnwidth = [15,15],
                header=dict(values=list(columns),
                align='left'),
                cells=dict(values= myarr,align='left'))
fig = go.Figure(data=tab)
fig.show()

In [31]:
display(HTML("<h4>4. Temps inter-déno. </h4>"))
mystr = "Temps Int-déno"
df_deno=df_str[df_str[mystr].notna()&(df_str[mystr]!=0)]
nb = len(df_deno)
if nb > 0:
    nb_avg = round(df_deno[mystr].mean())
    nb_med = round(df_deno[mystr].median())
    nb_var = round(df_deno[mystr].var())
    nb_min = df_deno[mystr].min()
    nb_max = df_deno[mystr].max()
    #
    ec_avg = round(df_deno["Ecart type - "+mystr].mean())
    ec_med = round(df_deno["Ecart type - "+mystr].median())
    ec_var = round(df_deno["Ecart type - "+mystr].var())
    ec_min = df_deno["Ecart type - "+mystr].min()
    ec_max = df_deno["Ecart type - "+mystr].max()
    #
    rp_avg = round(df_deno["Rang percentile - "+mystr].mean())
    rp_med = round(df_deno["Rang percentile - "+mystr].median())
    rp_var = round(df_deno["Rang percentile - "+mystr].var())
    rp_min = df_deno["Rang percentile - "+mystr].min()
    rp_max = df_deno["Rang percentile - "+mystr].max()
else:
    nb_avg = nb_med = nb_var = nb_min = nb_max = ec_avg = ec_med = ec_var = ec_min = ec_max = rp_avg= rp_med = rp_var = rp_min = rp_max=0
columns = ['',"Score", "Ecart-type", "Rang percentile"]

myarr = [['Effectif', 'Moyenne', 'Médiane', 'Variance', 'Min.', 'Max']]
myarr.append([nb, nb_avg, nb_med, nb_var, nb_min, nb_max])
myarr.append(['-',ec_avg, ec_med, ec_var, ec_min, ec_max])
myarr.append(['-',rp_avg, rp_med, rp_var, rp_min, rp_max])


tab =  go.Table(
                columnwidth = [15,15],
                header=dict(values=list(columns),
                align='left'),
                cells=dict(values= myarr,align='left'))
fig = go.Figure(data=tab)
fig.show()

In [23]:
display(HTML("<h3>Effectifs scores d'erreurs</h3>"))

In [24]:
nb_erreurs_nc = len(df_str[df_str["Nombre d'erreurs non corrigés (100 items)"]>0])
nb_erreurs_tps = len(df_str[df_str["Nombre d'erreurs (Temps de lecture)"]>0])
nb_erreurs_int = len(df_str[df_str["Nombre d’erreurs non corrigées (Temps interférence)"]>0])
nb_erreurs_int_deno = len(df_str[df_str["Nombre d’erreurs non corrigées (Int-déno)"]>0])

In [25]:
columns = ['',"Nombre d'erreurs non corrigées (100 items)", "Nombre d'erreurs (Temps de lecture)", "Nombre d'erreurs non corrigées (Temps interférence)", "Nombre d'erreurs non corrigées (Int-déno)"]

myarr = [['Effectif']]
myarr.append([nb_erreurs_nc])
myarr.append([nb_erreurs_tps])
myarr.append([nb_erreurs_int])
myarr.append([nb_erreurs_int_deno])



tab =  go.Table(
                columnwidth = [15,15],
                header=dict(values=list(columns),
                align='left'),
                cells=dict(values= myarr,align='left'))
fig = go.Figure(data=tab)
fig.show()

In [26]:
display(HTML("<h3>Scores d'erreurs</h3>"))

In [27]:
display(HTML("<h4>1. Nombre d'erreurs non corrigées (100 items)</h4>"))
display(HTML("<p>Toutes les valeurs sont nulles.</p>"))
display(HTML("<h4>2. Nombre d'erreurs (Temps de lecture)</h4>"))
display(HTML("<p>Toutes les valeurs sont nulles.</p>"))
display(HTML("<h4>3. Nombre d'erreurs non corrigées (Temps interférence)</h4>"))

In [28]:
df_deno=df_str[df_str["Nombre d’erreurs non corrigées (Temps interférence)"].notna()&(df_str["Nombre d’erreurs non corrigées (Temps interférence)"]!=0)]
nb = len(df_deno)
if nb > 0:
    nb_avg = round(df_deno["Nombre d’erreurs non corrigées (Temps interférence)"].mean())
    nb_med = round(df_deno["Nombre d’erreurs non corrigées (Temps interférence)"].median())
    nb_var = round(df_deno["Nombre d’erreurs non corrigées (Temps interférence)"].var())
    nb_min = df_deno["Nombre d’erreurs non corrigées (Temps interférence)"].min()
    nb_max = df_deno["Nombre d’erreurs non corrigées (Temps interférence)"].max()
    #
    ec_avg = round(df_deno["Ecart type - Nombre d’erreurs non corrigées (Temps interférence)"].mean())
    ec_med = round(df_deno["Ecart type - Nombre d’erreurs non corrigées (Temps interférence)"].median())
    ec_var = round(df_deno["Ecart type - Nombre d’erreurs non corrigées (Temps interférence)"].var())
    ec_min = df_deno["Ecart type - Nombre d’erreurs non corrigées (Temps interférence)"].min()
    ec_max = df_deno["Ecart type - Nombre d’erreurs non corrigées (Temps interférence)"].max()
    #
    rp_avg = round(df_deno["Rang percentile - Nombre d’erreurs non corrigées (Temps interférence)"].mean())
    rp_med = round(df_deno["Rang percentile - Nombre d’erreurs non corrigées (Temps interférence)"].median())
    rp_var = round(df_deno["Rang percentile - Nombre d’erreurs non corrigées (Temps interférence)"].var())
    rp_min = df_deno["Rang percentile - Nombre d’erreurs non corrigées (Temps interférence)"].min()
    rp_max = df_deno["Rang percentile - Nombre d’erreurs non corrigées (Temps interférence)"].max()
else:
    nb_avg = nb_med = nb_var = nb_min = nb_max = ec_avg = ec_med = ec_var = ec_min = ec_max = rp_avg= rp_med = rp_var = rp_min = rp_max=0
columns = ['',"Score", "Ecart-type", "Rang percentile"]

myarr = [['Effectif', 'Moyenne', 'Médiane', 'Variance', 'Min.', 'Max']]
myarr.append([nb, nb_avg, nb_med, nb_var, nb_min, nb_max])
myarr.append(['-',ec_avg, ec_med, ec_var, ec_min, ec_max])
myarr.append(['-',rp_avg, rp_med, rp_var, rp_min, rp_max])


tab =  go.Table(
                columnwidth = [15,15],
                header=dict(values=list(columns),
                align='left'),
                cells=dict(values= myarr,align='left'))
fig = go.Figure(data=tab)
fig.show()
display(HTML("<h4>4. Nombre d'erreurs non corrigées (int-déno)</h4>"))

In [29]:
df_deno=df_str[df_str["Nombre d’erreurs non corrigées (Int-déno)"].notna()&(df_str["Nombre d’erreurs non corrigées (Int-déno)"]!=0)]
nb = len(df_deno)
if nb > 0:
    nb_avg = round(df_deno["Nombre d’erreurs non corrigées (Int-déno)"].mean())
    nb_med = round(df_deno["Nombre d’erreurs non corrigées (Int-déno)"].median())
    nb_var = round(df_deno["Nombre d’erreurs non corrigées (Int-déno)"].var())
    nb_min = df_deno["Nombre d’erreurs non corrigées (Int-déno)"].min()
    nb_max = df_deno["Nombre d’erreurs non corrigées (Int-déno)"].max()
    #
    ec_avg = round(df_deno["Ecart type - Nombre d’erreurs non corrigées (Int-déno)"].mean())
    ec_med = round(df_deno["Ecart type - Nombre d’erreurs non corrigées (Int-déno)"].median())
    ec_var = round(df_deno["Ecart type - Nombre d’erreurs non corrigées (Int-déno)"].var())
    ec_min = df_deno["Ecart type - Nombre d’erreurs non corrigées (Int-déno)"].min()
    ec_max = df_deno["Ecart type - Nombre d’erreurs non corrigées (Int-déno)"].max()
    #
    rp_avg = round(df_deno["Rang percentile - Nombre d’erreurs non corrigées (Int-déno)"].mean())
    rp_med = round(df_deno["Rang percentile - Nombre d’erreurs non corrigées (Int-déno)"].median())
    rp_var = round(df_deno["Rang percentile - Nombre d’erreurs non corrigées (Int-déno)"].var())
    rp_min = df_deno["Rang percentile - Nombre d’erreurs non corrigées (Int-déno)"].min()
    rp_max = df_deno["Rang percentile - Nombre d’erreurs non corrigées (Int-déno)"].max()
else:
    nb_avg = nb_med = nb_var = nb_min = nb_max = ec_avg = ec_med = ec_var = ec_min = ec_max = rp_avg= rp_med = rp_var = rp_min = rp_max=0
columns = ['',"Score", "Ecart-type", "Rang percentile"]

myarr = [['Effectif', 'Moyenne', 'Médiane', 'Variance', 'Min.', 'Max']]
myarr.append([nb, nb_avg, nb_med, nb_var, nb_min, nb_max])
myarr.append(['-',ec_avg, ec_med, ec_var, ec_min, ec_max])
myarr.append(['-',rp_avg, rp_med, rp_var, rp_min, rp_max])


tab =  go.Table(
                columnwidth = [15,15],
                header=dict(values=list(columns),
                align='left'),
                cells=dict(values= myarr,align='left'))
fig = go.Figure(data=tab)
fig.show()